In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
from itertools import chain

In [7]:
# Obtenemos las columnas a buscar
columnas_usuario = [
                    ['ga:userType'], 
                    ['ga:browser', 'ga:operatingSystem'], 
                    ['ga:deviceCategory'], 
                    ['ga:mobileDeviceBranding', 'ga:mobileDeviceModel', 'ga:mobileInputSelector', 'ga:mobileDeviceInfo'], 
                    ['ga:subContinent', 'ga:country', 'ga:region', 'ga:city'], 
                    ['ga:javaEnabled', 'ga:language'],
                    ['ga:pagePath', 'ga:pageTitle']
                   ]

columnas_pagina = [
                   ['ga:pagePath', 'ga:pageTitle']
                  ]

# Buscamos valores unicos por reporte
id = pd.read_csv('id.csv')
j = len(columnas_usuario)
k = len(columnas_pagina)
z = 0
i = 0
lista_unicos_pagina = []
while i < k:
    for columna in columnas_pagina[i]:
        lista_unicos_pagina.append([columna])
    i +=1
i = 0
lista_unicos_usuarios = []
while i < j:
    for columna in columnas_usuario[i]:
        lista_unicos_usuarios.append([columna])
    i +=1
    
while z < len(id):
    print(z)
    VIEW_ID = str(id.iloc[z][3])
    titulo = str(id.iloc[z][0]) + '-' + str(id.iloc[z][1]) + '-' + str(id.iloc[z][2])
    lista_df_usuario = []
    i = 0
    while i < j:
        try:
            lista_df_usuario.append(pd.read_csv('produccion/usuario/' + str(z) + '_' + titulo + '_' + VIEW_ID + '/df_' + str(i) + '_final.csv'))
        except FileNotFoundError:
            pass
        i += 1
    lista_df_pagina = []
    i = 0
    while i < k + 1:
        try:
            lista_df_pagina.append(pd.read_csv('produccion/pagina/' + str(z) + '_' + titulo + '_' + str(i+1) + '.csv'))
        except FileNotFoundError:
            pass
        i += 1
        
    if lista_df_usuario != []:
        print('Archivo: ' + str(z) + '_' + titulo + ' (usuario)')
        
        # ga:userType
        lista_unicos_usuarios[0].append(lista_df_usuario[0][columnas_usuario[0][0]].unique().tolist())
        
        # ga:browser
        lista_unicos_usuarios[1].append(lista_df_usuario[1][columnas_usuario[1][0]].unique().tolist())
        
        # ga:operatingSystem
        lista_unicos_usuarios[2].append(lista_df_usuario[1][columnas_usuario[1][1]].unique().tolist())
        
        # ga:deviceCategory
        lista_unicos_usuarios[3].append(lista_df_usuario[2][columnas_usuario[2][0]].unique().tolist())
        
        # ga:mobileDeviceBranding
        lista_unicos_usuarios[4].append(lista_df_usuario[3][columnas_usuario[3][0]].unique().tolist())
        
        # ga:mobileDeviceModel
        lista_unicos_usuarios[5].append(lista_df_usuario[3][columnas_usuario[3][1]].unique().tolist())
        
        # ga:mobileInputSelector
        lista_unicos_usuarios[6].append(lista_df_usuario[3][columnas_usuario[3][2]].unique().tolist())
        
        # ga:mobileDeviceInfo
        lista_unicos_usuarios[7].append(lista_df_usuario[3][columnas_usuario[3][3]].unique().tolist())
        
        # ga:subContinent
        lista_unicos_usuarios[8].append(lista_df_usuario[4][columnas_usuario[4][0]].unique().tolist())
        
        # ga:country
        lista_unicos_usuarios[9].append(lista_df_usuario[4][columnas_usuario[4][1]].unique().tolist())
        
        # ga:region
        lista_unicos_usuarios[10].append(lista_df_usuario[4][columnas_usuario[4][2]].unique().tolist())
        
        # ga:city
        lista_unicos_usuarios[11].append(lista_df_usuario[4][columnas_usuario[4][3]].unique().tolist())
        
        # ga:javaEnabled
        lista_unicos_usuarios[12].append(lista_df_usuario[5][columnas_usuario[5][0]].unique().tolist())
        
        # ga:language
        lista_unicos_usuarios[13].append(lista_df_usuario[5][columnas_usuario[5][1]].unique().tolist())

        # ga:pagePath
        lista_unicos_usuarios[14].append(lista_df_usuario[6][columnas_usuario[6][0]].unique().tolist())
        
        # ga:pageTitle
        lista_unicos_usuarios[15].append(lista_df_usuario[6][columnas_usuario[6][1]].unique().tolist())
        
    else:
        print('Archivo: ' + str(z) + '_' + titulo + ' (usuario) no tiene vistas registradas')
        
    if lista_df_pagina != []:
        print('Archivo: ' + str(z) + '_' + titulo + ' (pagina)')
        
        #ga:pagePath
        lista_unicos_pagina[0].append(lista_df_pagina[0][columnas_pagina[0][0]].unique().tolist())
        
        #ga:pageTitle
        lista_unicos_pagina[1].append(lista_df_pagina[0][columnas_pagina[0][1]].unique().tolist())
    else:
        print('Archivo: ' + str(z) + '_' + titulo + ' (pagina)' + ' no tiene vistas registradas')    

    z += 1

print('Filtrando...')
i = 0
lista_pagina = []
while i < k:
    for columna in columnas_pagina[i]:
        lista_pagina.append(set())
    i +=1
i = 0
lista_usuarios = []
while i < j:
    for columna in columnas_usuario[i]:
        lista_usuarios.append(set())
    i +=1   

i = 0
while i < len(lista_usuarios):
    lista_unicos_usuarios[i].pop(0)
    for x in list(chain(*lista_unicos_usuarios[i])):
        lista_usuarios[i].add(x)
    i += 1
    
i = 0
while i < len(lista_pagina):
    lista_unicos_pagina[i].pop(0)
    for x in list(chain(*lista_unicos_pagina[i])):
        lista_pagina[i].add(x)
    i += 1
    
i = 0
lista_nombres_usuarios = []
while i < j:
    for columna in columnas_usuario[i]:
        lista_nombres_usuarios.append([columna])
    i +=1

print('Guardando...')
df_usuario = pd.DataFrame(list([]))
i = 0
while i < len(lista_usuarios): 
    df = pd.DataFrame(lista_usuarios[i], columns=lista_nombres_usuarios[i])
    df_usuario = df_usuario.merge(df, left_index=True, right_index=True, how='outer')
    i += 1
    
df_usuario.to_csv('produccion/unique_usuario.csv', encoding='utf8')

i = 0
lista_nombres_pagina = []
while i < k:
    for columna in columnas_pagina[i]:
        lista_nombres_pagina.append([columna])
    i +=1
    

df_pagina = pd.DataFrame(list([]))
i = 0
while i < len(lista_pagina): 
    df = pd.DataFrame(lista_pagina[i], columns=lista_nombres_pagina[i])
    df_pagina = df_pagina.merge(df, left_index=True, right_index=True, how='outer')
    i += 1
    
df_pagina.to_csv('produccion/unique_pagina.csv', encoding='utf8')

0
Archivo: 0_educatic_CTE-becarios-becarios (usuario) no tiene vistas registradas
Archivo: 0_educatic_CTE-becarios-becarios (pagina) no tiene vistas registradas
1
Archivo: 1_educatic_CTE-Becarios_formatic-Becarios_h@bitat (usuario)
Archivo: 1_educatic_CTE-Becarios_formatic-Becarios_h@bitat (pagina)
2
Archivo: 2_educatic_CTE-Chat_de_ayuda-Todos_los_datos_del_sitio_web (usuario) no tiene vistas registradas
Archivo: 2_educatic_CTE-Chat_de_ayuda-Todos_los_datos_del_sitio_web (pagina) no tiene vistas registradas
3
Archivo: 3_educatic_CTE-Consulta_ENP_Diplo_emi_1_y_2-All_Web_Site_Data (usuario) no tiene vistas registradas
Archivo: 3_educatic_CTE-Consulta_ENP_Diplo_emi_1_y_2-All_Web_Site_Data (pagina) no tiene vistas registradas
4
Archivo: 4_educatic_CTE-Dip_Aplicaciones_TIC_a_la_FCPyS-Dip_Aplicaciones_TIC_a_la_FCPyS (usuario) no tiene vistas registradas
Archivo: 4_educatic_CTE-Dip_Aplicaciones_TIC_a_la_FCPyS-Dip_Aplicaciones_TIC_a_la_FCPyS (pagina) no tiene vistas registradas
5
Archivo: 5_ed

Archivo: 47_TICometro-Retos_TIC-Todos_los_datos_del_sitio_web (usuario)
Archivo: 47_TICometro-Retos_TIC-Todos_los_datos_del_sitio_web (pagina)
48
Archivo: 48_TICometro-TICometro-TICometro (usuario)
Archivo: 48_TICometro-TICometro-TICometro (pagina)
49
Archivo: 49_TICOMETRO-TICOMETRO-Todos_los_datos_del_sitio_web (usuario) no tiene vistas registradas
Archivo: 49_TICOMETRO-TICOMETRO-Todos_los_datos_del_sitio_web (pagina) no tiene vistas registradas
Filtrando...
Guardando...
